In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00


In [2]:
!pip install --upgrade transformers

In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/ozcangundes/multitask-question-generation.git

Cloning into 'multitask-question-generation'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 50 (delta 21), reused 23 (delta 5), pack-reused 0
Receiving objects: 100% (50/50), 18.00 KiB | 18.00 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [5]:
%cd multitask-question-generation/

/content/multitask-question-generation


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("ozcangundes/mt5-multitask-qa-qg-turkish")
model = AutoModelForSeq2SeqLM.from_pretrained("ozcangundes/mt5-multitask-qa-qg-turkish")

from pipelines import pipeline #pipelines.py script in the cloned repo
multimodel = pipeline("multitask-qa-qg",tokenizer=tokenizer,model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')

path = '/content/drive/MyDrive/jsonpars'

file_lists = os.listdir(path)
json_list = []
for file_ in file_lists:
    if file_.endswith('.json'):
        json_list.append(file_)

print(json_list)

Mounted at /content/drive
['test_data.json', 'train_data_2.json', 'test_data_2.json', 'train_data.json', 'train_data_1.json', 'train_data_v2.json']


In [ ]:
# id-1 = json
# id-2 = generator

import pandas as pd

json_qa_df = pd.DataFrame(columns=['Title', 'Context', "id", 'Question', 'Answer'])

for json_file in json_list:
    json_data = pd.read_json(path+'/'+json_file)
    
    for data in json_data['data']:
        title = data['title']
        context = data['paragraphs'][0]['context']

        for qa in data['paragraphs'][0]['qas']:
            id_ = 1  
            question = qa['question']
            answer = qa['answers'][0]['text']

            json_qa_df.loc[len(json_qa_df)] = [title, context, id_, question, answer]

In [9]:
df_qst = [i for i in json_qa_df['Question']]
df_ans = [i for i in json_qa_df['Answer']]

generated_qa_df = pd.DataFrame(columns=['Title', 'Context', "id", 'Question', 'Answer'])

for context in json_qa_df['Context'].unique():

  generator_qa = multimodel(context)

  for qa in range(0, len(generator_qa)):

    if (generator_qa[qa]['question'] not in df_qst) or (generator_qa[qa]['answer'] not in df_ans):

      generated_qa_df.loc[len(generated_qa_df)] = [json_qa_df[json_qa_df['Context'] == context ]['Title'].iloc[0], context, 2, generator_qa[qa]['question'], generator_qa[qa]['answer']]


In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
generated_and_json_qa = pd.concat([json_qa_df, generated_qa_df ], ignore_index=True)

In [ ]:
qst_1 = generated_and_json_qa[generated_and_json_qa['id'] == 1]['Question'].tolist()
ans_1 = generated_and_json_qa[generated_and_json_qa['id'] == 1]['Answer'].tolist()
qst_2 = generated_and_json_qa[generated_and_json_qa['id'] == 2]['Question'].tolist()
ans_2 = generated_and_json_qa[generated_and_json_qa['id'] == 2]['Answer'].tolist()

a = 0 

for q1, a1 in zip(qst_1, ans_1):
    for q2, a2 in zip(qst_2, ans_2):
        if q1 == q2 and a1 == a2:
            a += 1

print(a)

In [16]:
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

qa_bleu_score_df = pd.DataFrame(columns=['Json-q', 'Generator-q', 'Answer', 'Bleu Score'])

for i in range(0, len(generated_and_json_qa)):

    if generated_and_json_qa.iloc[i]['id'] == 1:

        for a in range(0, len(generated_and_json_qa)):

            if generated_and_json_qa.iloc[a]['id'] == 2 and (generated_and_json_qa.iloc[i]['Answer'] == generated_and_json_qa.iloc[a]['Answer']):

                ref_question = generated_and_json_qa.iloc[i]['Question'].split()
                gen_question = generated_and_json_qa.iloc[a]['Question'].split()
                smoothing = SmoothingFunction().method3
                bleu_score = sentence_bleu([ref_question], gen_question, smoothing_function=smoothing)

                qa_bleu_score_df.loc[len(qa_bleu_score_df)] = [generated_and_json_qa.iloc[i]['Question'], generated_and_json_qa.iloc[a]['Question'], generated_and_json_qa.iloc[i]['Answer'], bleu_score]

In [ ]:
data_bleu_score = pd.DataFrame(columns=['Json Question', 'Generator Question', 'Answer', 'Bleu Score'])

for answer in qa_bleu_score_df['Answer'].unique():

    max_bleu = qa_bleu_score_df[qa_bleu_score_df['Answer'] == answer]['Bleu Score'].max()

    json_q = qa_bleu_score_df[(qa_bleu_score_df['Answer'] == answer) & (qa_bleu_score_df['Bleu Score'] == max_bleu)]['Json-q'].values[0]

    generator_q = qa_bleu_score_df[(qa_bleu_score_df['Answer'] == answer) & (qa_bleu_score_df['Bleu Score'] == max_bleu)]['Generator-q'].values[0]


    data_bleu_score.loc[len(data_bleu_score)] = [json_q, generator_q, answer, max_bleu]

In [ ]:
data_bleu_score.to_csv('data_bleu_score.csv', index=False)

In [1]:
import pandas as pd

data_bleu_score = pd.read_csv('data_bleu_score.csv')
data = data_bleu_score.copy()
data.head()

,Json Question,Generator Question,Answer,Bleu Score
0,II. Mehmed'nin Avrupa'da tanınan adı nedir?,Fatih Sultan Mehmed'in Avrupa'da tanınan adı n...,Grand Turco 'Büyük Türk' veya Turcarum Imperat...,0.411134
1,II. Mehmed kaç yıl boyunca hüküm sürdü?,II. Mehmed ilk olarak kaç yıl boyunca hüküm sü...,30 yıl boyunca,0.513345
2,II. Mehmed İstanbul'u kaç yaşında fethetti?,II. Mehmed kaç yaşında İstanbul'u fethetti?,21 yaşında,0.254066
3,İstanbul'un fethi ile hangi çağın sonununu get...,"II. Mehmed, 21 yaşında İstanbul'u fethetek 100...",Orta Çağ'ın sonu,0.031974
4,İstanbul'un fethi ile hangi çağın başlangıçı o...,II. Mehmed Orta Çağ'ın sonu ne olarak kabul ed...,Yeni Çağ'ın başlangıcı,0.177474


In [2]:
len(data)

621

In [2]:
data.duplicated().any()

False

In [6]:
length = len(data)
bad_bleu = len(data[data['Bleu Score'] < 0.1])
average_bleu = len(data[(data['Bleu Score'] >= 0.1) & (data['Bleu Score'] <= 0.33)])
good_bleu = len(data[data['Bleu Score'] >= 0.34])

print('Rate of bad bleu score %' + str(bad_bleu/length * 100))
print('Rate of average bleu score %' + str(average_bleu/length * 100))
print('Rate of good bleu score %' + str(good_bleu/length * 100))

Rate of bad bleu score %14.492753623188406
Rate of average bleu score %20.933977455716587
Rate of good bleu score %64.4122383252818
